In [1]:
import sagemaker
import boto3

In [2]:
sess = sagemaker.Session()

In [34]:
role = "SageMakerRole"

In [4]:
sagemaker_session_bucket="loyalhackathon"

In [5]:
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: SageMakerRole
sagemaker bucket: loyalhackathon
sagemaker session region: us-east-1


# Load model from the hub

In [12]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Set the name of the model to load from the Hugging Face model hub
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'


In [13]:
# Load the model and tokenizer from the Hugging Face model hub
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [14]:
save_dir = f'./models/{model_name}'

In [15]:
# Save the model and tokenizer to the specified directory
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('./models/distilbert-base-uncased-finetuned-sst-2-english/tokenizer_config.json',
 './models/distilbert-base-uncased-finetuned-sst-2-english/special_tokens_map.json',
 './models/distilbert-base-uncased-finetuned-sst-2-english/vocab.txt',
 './models/distilbert-base-uncased-finetuned-sst-2-english/added_tokens.json',
 './models/distilbert-base-uncased-finetuned-sst-2-english/tokenizer.json')

# Compress model

In [16]:
import tarfile

In [19]:

# Set the path to the directory containing the saved model files
model_dir = f'./models/{model_name}'

# Set the path and name of the output tar file
output_file = './compressed_models/model.tar.gz'

In [20]:
# Create the tar file
with tarfile.open(output_file, mode='w:gz') as archive:
    archive.add(model_dir, arcname='.')

## Upload compressed file to S3 bucket

In [21]:
import boto3

# Set up the S3 client and bucket name
s3 = boto3.client('s3')
bucket_name = 'loyalhackathon'

In [56]:
# Set the path and name of the input tar file
input_file = './compressed_models/model.tar.gz'
object_key = 'model_2.tar.gz'

In [57]:
# Upload the file to the S3 bucket
s3.upload_file(input_file, bucket_name, object_key)

# Train model

In [58]:
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.serverless import ServerlessInferenceConfig

In [59]:
role = "SageMakerRole"

In [75]:
# Hub Model configuration. <https://huggingface.co/models>
hub = {
    'HF_MODEL_ID':'distilbert-base-uncased-finetuned-sst-2-english',
    'HF_TASK':'text-classification'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=f"s3://{bucket_name}/{object_key}",
   role="SageMakerRole",         # iam role with permissions to create an Endpoint
   transformers_version="4.12",  # transformers version used
   pytorch_version="1.9",        # pytorch version used
   py_version='py38',            # python version used
)

In [76]:
# Specify MemorySizeInMB and MaxConcurrency in the serverless config object
serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=3072, max_concurrency=1,
)

In [78]:
predictor = huggingface_model.deploy(
    serverless_inference_config=serverless_config,
    endpoint_name="huggingface-endpoint"
)

---!

In [45]:
print(predictor)

In [44]:
data = {
  "inputs": "I potatoes"
}

res = predictor.predict(data=data)
print(res)

[{'label': 'NEGATIVE', 'score': 0.9482402205467224}]


In [79]:
predictor.delete_endpoint()

# Delete SageMaker Endpoint

In [1]:
import boto3

In [2]:
# Initialize a Boto3 SageMaker client
sagemaker = boto3.client('sagemaker')

In [8]:
# List all existing endpoints
response = sagemaker.list_endpoints()

In [9]:
# Print the endpoint names and status
for endpoint in response['Endpoints']:
    print(endpoint['EndpointName'], endpoint['EndpointStatus'])

john-endpoint-test InService
regroup-endpoint InService
intent-endpoint-2 InService
huggingface-endpoint InService


In [10]:
# Specify the endpoint name(s) that you want to delete
endpoints_to_delete = ['john-endpoint-test']

In [11]:
# Loop through the endpoints and delete them
for endpoint in endpoints_to_delete:
    try:
        sagemaker.delete_endpoint(EndpointName=endpoint)
        print(f"Deleted endpoint: {endpoint}")
    except:
        print(f"Failed to delete endpoint: {endpoint}")

Deleted endpoint: john-endpoint-test
